In [ ]:
import ast
import astunparse as astu
import glob
import pandas as pd

from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [ ]:
def clone(repo,dest='drive/MyDrive/Thesis2/repos/'):
  f = 'https://github.com/'+repo+ '.git'
  p = dest + repo.split('/')[-1].strip()
  !GIT_TERMINAL_PROMPT=0 git clone --depth 1 $f $p

In [ ]:
def extractMethodsAST(filename,outputdir="drive/MyDrive/Thesis2/snippets/",count = 0):
  try: 
    r = open(filename,'r')
    t = ast.parse(r.read())
    imports = []
    methods = []
    for node in ast.walk(t):
      if isinstance(node,ast.ImportFrom) or isinstance(node,ast.Import):
        imports.append(astu.unparse(node).strip())
      if isinstance(node,ast.FunctionDef):
        methods.append(astu.unparse(node))
      if isinstance(node,ast.If) and "if (__name__ == '__main__')" in astu.unparse(node):
        methods.append(astu.unparse(node))
    if len(methods) == 0:
      snippetfile = f"{outputdir}snippet{str(count)}.py"
      print(snippetfile)
      o = open(snippetfile,"w")
      o.write(astu.unparse(t))
      o.close()

    imp = '\n'.join(imports) + '\n'
    for i in methods:
      count+=1
      snippetfile = f"{outputdir}snippet{str(count)}.py"
      print(snippetfile)
      o = open(snippetfile,"w")
      o.write(imp+i)
      o.close()

    return count
  except (SyntaxError,UnicodeDecodeError,IsADirectoryError,AttributeError,FileNotFoundError) as error: 
    return count

In [ ]:
def getSnippets(dir,snippets):
  print(str(i) + '. ' + dir)
  files = []
  s = '/*.py'
  sf = '/'
  c= '/*'
  count = 0
  while True:
    f = glob.glob(dir+s)
    f1 = glob.glob(dir+sf)
    if f==[] and f1==[] and count != 0:
      break
    for file in f:
      files.append(file)
      snippets = extractMethodsAST(file,"drive/MyDrive/Thesis2/snippets/",snippets)
    sf=c+sf
    s=c+s
    count+=1
  return snippets

In [ ]:
df = pd.read_csv('drive/MyDrive/Thesis2/marked_dataset.csv',index_col=0)
print(df.shape)

Mounted at /content/drive
(64009, 26)


In [ ]:
# for each repo in the dataframe:
  # clone repo
  # extract files
    # create snippets from each file
  # delete repo
  # mark completed
  # save updated dataframe

dir = "drive/MyDrive/Thesis2/"
snippets = 0
for i in range(df.shape[0]):
  repo = df.iloc[i][0]
  
  if df.iloc[i]['complete'] == False:
    clone(repo)
    repodir = f"{dir}repos/{repo.split('/')[-1]}"
    try:
      snippets = getSnippets(repodir,snippets)
    except RecursionError as error:
      print(error)
    !rm -rf $repodir
    df.at[i,'complete'] = True
    df.to_csv(dir+'marked_dataset.csv')